This is the first post in a series I hope to write on a new clustering algorithm in Python. The model is essentially a Gaussian Mixture Model, so I will be comparing it with the [GaussianMixture](http://scikit-learn.org/stable/modules/mixture.html#mixture) algorithm in `scikit-learn`. My algorithm will be different, using Markov Chain Monte Carlo, and I have already implemented it in C++. I hope it will be quite accurate, estimating the number of clusters at least as accurately as existing methods, but I will have to sacrifice some scalability as a result. A visualization of my C++ algorithm is [online here](https://aaronmcdaid.github.io/demos/mvnMCMC/mvnMCMC.html) but I'm going to write this in Python as it will be more accessible via Python, and it will be a good project to help me dive deeply into Python and Jupyter notebooks and so on

However, this first post is going to be quite small and simple. I hope to blog regular, small, posts on this topic as I make progress. I have multiple motivations for this: transition to Python 3, investigate existing clustering algorithms in Python, learn how to make nice Jupyter notebooks (including animations perhaps). Please give me feedback on everything!

## Calculating the sample mean (centroid) in a cluster efficiently and accurately

I will focus on one simple, but important, issue in this first post. We have one-dimensional data wth three datapoints at 300,2.71,2.70. First we compute the sample mean:

In [75]:
import numpy as np
x=314.123
y=2.71
z=2.70

print( (x+y+z)/3 )
print( np.mean( [x,y,z] ))

106.51099999999998
106.51099999999998


As the algorithm proceeds, it will add and remove datapoints from clusters. We need to keep track of how the sample mean changes as we add and remove data points. It seems natural to keep a `total` variable and then add and remove the values to it and from it:

In [76]:
total = 0.0
print("initial total, empty cluster:\t", total)
total += x
print("total after adding 'x':\t\t", total)
total += y
print("total after adding 'y':\t\t", total)
total += z
print("total after adding 'z':\t\t", total)
total -= x
print("total after subtracting 'x':\t\t", total)
total -= y
print("total after subtracting 'y':\t\t", total)
total -= z
print("total after subtracting 'z':\t\t", total)


initial total, empty cluster:	 0.0
total after adding 'x':		 314.123
total after adding 'y':		 316.83299999999997
total after adding 'z':		 319.53299999999996
total after subtracting 'x':		 5.409999999999968
total after subtracting 'y':		 2.699999999999968
total after subtracting 'z':		 -3.197442310920451e-14


Note the final value. It's not exactly zero. But it should be zero. We've adding three values, and then subtracted all three values, so the final total should be zero.

(More explanation of this issue is [here on Wikipedia](https://en.wikipedia.org/wiki/Floating-point_arithmetic#Accuracy_problems))

Usually, this error isn't a problem, as the error is quite small. But in the algorithm I plan to develop here, datapoints will be added and removed repeatedly and we need to be careful as the error can build up. To see how it can build up, consider this generator:

In [77]:
import more_itertools as it

def add_and_remove_forever():
    total = 0.0
    while True:
        total += x
        total += y
        total += z
        total -= x
        total -= y
        total -= z
        yield total
        
it.take( 10, add_and_remove_forever() )

[-3.197442310920451e-14,
 -8.881784197001252e-14,
 -1.4566126083082054e-13,
 -2.0250467969162855e-13,
 -2.5934809855243657e-13,
 -3.161915174132446e-13,
 -3.730349362740526e-13,
 -4.298783551348606e-13,
 -4.867217739956686e-13,
 -5.435651928564766e-13]

Notice how it is moving further from zero. The last entry in that list is after a total of 60 arithmetic operations (30 additions interleaved with 30 subtrations). The error continues to build up. It's very slow nowadays, as we are using 64-bit floating point numbers (`double` in C or C++). If we used 32-bit floating point numbers (`float` in C or C++) then the error would grow much more quickly.

Even though the error grows slowly (under 64-bits) we might still want a more stable system. It's highly desirable that when a cluster returns to the same state as it had previously, that the `total` has exactly the same value. This would help us to verify some correctness properties in the algorithm.

The obvious (but slow) solution is to simply recompute the full total from scratch each time, instead of remembering and modifying a `total` variable.

In [87]:
class slow_recompute:
    def __init__(self, values):
        self.values = values
        self.N = len(values)
        self.is_a_member = [ False for i in range(self.N)]
    def add_index(self, idx):
        self.is_a_member[idx] = True
    def rem_index(self, idx):
        self.is_a_member[idx] = False
    def get_total(self):
        return np.sum( self.values[i] for i in range(self.N) if self.is_a_member[i])
        

sr = slow_recompute([x,y,z])

# Now add the three elements in turn:
sr.add_index(0) # add the first item, in this case 'x=314.123'
print(sr.get_total())
sr.add_index(1) # add the first item, in this case 'y=2.71'
print(sr.get_total())
sr.add_index(2) # add the first item, in this case 'z=2.70'
print(sr.get_total())

# .. then remove them:
sr.rem_index(0) # remove the first item, in this case 'x=314.123'
print(sr.get_total())
sr.rem_index(1) # remove the first item, in this case 'y=2.71'
print(sr.get_total())
sr.rem_index(2) # remove the first item, in this case 'z=2.70'
print(sr.get_total())

314.123
316.83299999999997
319.53299999999996
5.41
2.7
0


As you would expect, the final answer you can see there is exactly zero. But this is slow, as it has to iterate over all the items in the list every time we call `get_total`.